In [47]:
import pandas as pd

# Load the uploaded files
folder = '/Users/kylewong/Downloads/demand-forecasting-kernels-only'
train_path = folder + '/train.csv'
test_path = folder + '/test.csv'
sample_submission_path = folder + '/sample_submission.csv'

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
sample_submission_df = pd.read_csv(sample_submission_path)

# Display basic information about the datasets
train_info = train_df.info()
test_info = test_df.info()
sample_submission_info = sample_submission_df.info()

train_df.head(), train_info, test_df.head(), test_info, sample_submission_df.head(), sample_submission_info


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   date       913000 non-null  datetime64[ns]
 1   store      913000 non-null  int64         
 2   item       913000 non-null  int64         
 3   sales      913000 non-null  int64         
 4   year_week  913000 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 34.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      45000 non-null  int64 
 1   date    45000 non-null  object
 2   store   45000 non-null  int64 
 3   item    45000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 2 columns):
 #

(        date  store  item  sales year_week
 0 2013-01-01      1     1     13    2013-1
 1 2013-01-02      1     1     11    2013-1
 2 2013-01-03      1     1     14    2013-1
 3 2013-01-04      1     1     13    2013-1
 4 2013-01-05      1     1     10    2013-1,
 None,
    id        date  store  item
 0   0  2018-01-01      1     1
 1   1  2018-01-02      1     1
 2   2  2018-01-03      1     1
 3   3  2018-01-04      1     1
 4   4  2018-01-05      1     1,
 None,
    id  sales
 0   0     52
 1   1     52
 2   2     52
 3   3     52
 4   4     52,
 None)

In [48]:
# Convert 'date' column to datetime format for easier manipulation
train_df['date'] = pd.to_datetime(train_df['date'])

# Add a 'week' column representing the year and week number
train_df['year_week'] = train_df['date'].dt.isocalendar().year.astype(str) + '-' + train_df['date'].dt.isocalendar().week.astype(str)

# Aggregate sales by store, item, and week
weekly_sales = train_df.groupby(['store', 'item', 'year_week'], as_index=False).agg({'sales': 'sum'})

# Aggregate sales across all stores for each item and week
aggregated_sales = weekly_sales.groupby(['item', 'year_week'], as_index=False).agg({'sales': 'sum'})

# Pivot the data to create a time-series format for each item
pivot_data = aggregated_sales.pivot(index='item', columns='year_week', values='sales').fillna(0)



In [49]:
# Prepare data for RNN with 12 weeks as input
time_steps = 12  # Using 12 weeks as input
X = []
y = []

for i in range(scaled_data.shape[1] - time_steps - 8):  # Exclude the last 8 weeks for prediction
    X.append(scaled_data[:, i:i + time_steps])  # Last 12 weeks as input
    y.append(scaled_data[:, i + time_steps:i + time_steps + 8].T)  # Next 8 weeks as output

X = np.array(X).reshape(-1, time_steps, 1)  # Shape: [samples, time_steps, features]
y = np.array(y).reshape(-1, 8)  # Shape: [samples, output_steps]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [50]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
import math

# Define the learning rate scheduler to start with a higher learning rate
def lr_schedule(epoch):
    initial_lr = 1e-2
    decay_rate = 0.1
    decay_epoch = 10  # After 10 epochs, reduce learning rate
    return initial_lr * math.pow(decay_rate, math.floor(epoch / decay_epoch))

# Define the RNN model
model = Sequential([
    LSTM(64, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(32, activation='relu', return_sequences=False),
    Dense(8)  # Predict the next 8 weeks
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-2), loss='mse', metrics=['mae'])

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5, verbose=1)
lr_scheduler = LearningRateScheduler(lr_schedule, verbose=1)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stop, reduce_lr, lr_scheduler],
    verbose=1
)

# Save the trained model
model.save('rnn_sales_forecast_model_12_weeks_with_lr_schedule.h5')

print("Model trained and saved as 'rnn_sales_forecast_model_12_weeks_with_lr_schedule.h5'.")


Epoch 1: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/50


2024-12-01 00:09:34.677238: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-12-01 00:09:34.891299: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_14.


302/302 [==============================] - ETA: 0s - loss: 0.0930 - mae: 0.2631

2024-12-01 00:11:41.752158: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


302/302 [==============================] - 132s 431ms/step - loss: 0.0930 - mae: 0.2631 - val_loss: 0.0883 - val_mae: 0.2571 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.01.
Epoch 2/50
302/302 [==============================] - 130s 432ms/step - loss: 0.0871 - mae: 0.2564 - val_loss: 0.0884 - val_mae: 0.2587 - lr: 0.0100

Epoch 3: LearningRateScheduler setting learning rate to 0.01.
Epoch 3/50
302/302 [==============================] - 128s 424ms/step - loss: 0.0867 - mae: 0.2566 - val_loss: 0.0871 - val_mae: 0.2572 - lr: 0.0100

Epoch 4: LearningRateScheduler setting learning rate to 0.01.
Epoch 4/50
302/302 [==============================] - 128s 424ms/step - loss: 0.0856 - mae: 0.2539 - val_loss: 0.0862 - val_mae: 0.2536 - lr: 0.0100

Epoch 5: LearningRateScheduler setting learning rate to 0.01.
Epoch 5/50
302/302 [==============================] - 128s 425ms/step - loss: 0.0852 - mae: 0.2515 - val_loss: 0.0858 - val_mae: 0.2540 - lr: 0.0100

Epoch 6: Lear

In [57]:

# Load the train data
train_data = pd.read_csv(train_path)
train_data['date'] = pd.to_datetime(train_data['date'])
train_data['year_week'] = train_data['date'].dt.isocalendar().year.astype(str) + '-' + train_data['date'].dt.isocalendar().week.astype(str)

# Aggregate weekly sales for each item
weekly_sales = train_data.groupby(['item', 'year_week'], as_index=False).agg({'sales': 'sum'})

# Pivot data to create a time-series format
pivot_data = weekly_sales.pivot(index='item', columns='year_week', values='sales').fillna(0)

# Scale the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(pivot_data)

# Extract the last 12 weeks for each item
last_12_weeks = scaled_data[:, -12:]

# Predict the next 4 weeks of sales
predicted_scaled = model.predict(last_12_weeks.reshape(last_12_weeks.shape[0], 12, 1))

# Scale back the predictions using MinMaxScaler parameters
predicted_sales = predicted_scaled * (scaler.data_max_[-1] - scaler.data_min_[-1]) + scaler.data_min_[-1]

# Extract the first 4 weeks of predictions
predicted_sales = predicted_sales[:, :4]

# Create the submission DataFrame
submission = pd.DataFrame(predicted_sales, columns=[
    'sales_forecast_week_1', 'sales_forecast_week_2', 'sales_forecast_week_3', 'sales_forecast_week_4'
])
submission.insert(0, 'item', pivot_data.index)

# Save the submission file
submission.to_csv('submission.csv', index=False)

print("Submission file 'submission.csv' created successfully.")


2/2 [==============================] - 0s 40ms/step
Submission file 'submission.csv' created successfully.


array([[0.05627706, 0.04761905, 0.06060606, ..., 0.04761905, 0.02597403,
        0.03896104],
       [0.04761905, 0.06060606, 0.05627706, ..., 0.02597403, 0.03896104,
        0.05627706],
       [0.06060606, 0.05627706, 0.04329004, ..., 0.03896104, 0.05627706,
        0.04761905],
       ...,
       [0.37662338, 0.44155844, 0.38095238, ..., 0.23809524, 0.27705628,
        0.31601732],
       [0.44155844, 0.38095238, 0.39393939, ..., 0.27705628, 0.31601732,
        0.34632035],
       [0.38095238, 0.39393939, 0.37662338, ..., 0.31601732, 0.34632035,
        0.3030303 ]])